# Python для анализа данных

## Pandas (join и merge)

*Ян Пиле, НИУ ВШЭ*

Давайте представим, что мы аналитики в компании, которая производит и продает скрепки. Нам нужно составить несколько отчетов для отдела продаж, чтобы посмотреть, как у них дела. Данные лежат в следующих словарях:

In [3]:
import numpy as np
import pandas as pd
# Dataframe of number of sales made by an employee
sales = {'Tony': 103,
         'Sally': 202,
         'Randy': 380,
         'Ellen': 101,
         'Fred': 82
        }
# Dataframe of all employees and the region they work in
region = {'Tony': 'West',
          'Sally': 'South',
          'Carl': 'West',
          'Archie': 'North',
          'Randy': 'East',
          'Ellen': 'South',
          'Fred': np.nan,
          'Mo': 'East',
          'HanWei': np.nan,
         }

Мы можем создать два отдельных dataframe из словарей 

In [4]:
# Make dataframes
sales_df = pd.DataFrame.from_dict(sales, orient='index', 
                                  columns=['sales'])
region_df = pd.DataFrame.from_dict(region, orient='index', 
                                   columns=['region'])

In [5]:
sales_df

,sales
Tony,103
Randy,380
Sally,202
Ellen,101
Fred,82


In [6]:
region_df

,region
Archie,North
HanWei,NaN
Randy,East
Ellen,South
Carl,West
Tony,West
Mo,East
Sally,South
Fred,NaN


Теперь давайте объединим все наши данные в один Датафрейм. Но как нам это сделать?
Датафреймы Pandas имеют много SQL-подобных функций. Иногда не до конца понятно, использовать join или merge. Их часто используют взаимозаменяемо (выбирая то, что пришло в голову первым). Так когда же мы должны использовать каждый из этих методов, и насколько точно они отличаются друг от друга? Попробуем разобраться.

### Join
Давайте начнем с join, потому что он самый простой. У датафреймов данных есть параметр index. Это ключ вашей таблицы, и если мы знаем индекс, то мы можем легко получить строку, содержащую наши данные, используя .loc. Если вы напечатаете свой датафрейм, вы увидите индекс в крайнем левом столбце. Еще его можно получить, напрямую использовав .index:

In [7]:
sales_df.index

Index([u'Tony', u'Randy', u'Sally', u'Ellen', u'Fred'], dtype='object')

Значит индекс в sales_df - это имя продавца. Кстати, в отличие от первичного ключа таблицы SQL, индекс датафрейма не обязательно должен быть уникальным. Но уникальный индекс делает нашу жизнь проще, а время, необходимое для поиска в нашем датафрейме, короче. Учитывая индекс, мы можем найти данные строки так:

In [8]:
sales_df.loc['Tony']

sales    103
Name: Tony, dtype: int64

Вернемся к join'ам. Метод join берет два датафрейма и соединяет их по индексам (технически вы можете выбрать столбец для объединения для левого датафрейма). Давайте посмотрим, что происходит, если мы объединим(aka сджойним) наши два датафрейма с помощью метода join:

In [9]:
joined_df = region_df.join(sales_df, how='left')
print(joined_df)

       region  sales
Archie  North    NaN
HanWei    NaN    NaN
Randy    East  380.0
Ellen   South  101.0
Carl     West    NaN
Tony     West  103.0
Mo       East    NaN
Sally   South  202.0
Fred      NaN   82.0


Результат выглядит как результат SQL-join'a (по сути, это почти то же самое). 

Метод join использует индекс или указанный столбец из левого датафрейма в качестве ключа джойна. Таким образом, столбец, по которому мы джойним левый датафрейм, не обязательно должен быть его индексом. Но вот для правильного датафрейма ключ джойна должен быть его индексом ОБЯЗАТЕЛЬНО. 

Лично мне проще воспринимать метод join как объединение на основе индекса и использовать merge, если не хочется привязываться к индексам.

В объединенном датафрейме есть несколько NaN. Так произошло , потому что не у всех сотрудников были продажи. Те, у кого не было продаж, отсутствуют в sales_df, но мы по-прежнему отображаем их, потому что мы выполнили left join (указав «how = left»), которое возвращает все строки из левого датафрейма region_df, независимо от того, есть ли совпадение в правом. Если мы не хотим отображать какие-либо NaN в нашем результате соединения, мы вместо этого можем сделать inner join (указав «how = inner»).

### Merge

На базовом уровне Merge делает более или менее то же самое, что и join. Оба метода используются для объединения двух датафреймов, но merge является более универсальным за счет более подробного описания входных данных. Давайте посмотрим, как мы можем создать тот же обхединенный датафрейм с помощью merge:

In [10]:
joined_df_merge = region_df.merge(sales_df, how='left', 
                                      left_index=True,
                                      right_index=True)
print(joined_df_merge)

       region  sales
Archie  North    NaN
HanWei    NaN    NaN
Randy    East  380.0
Ellen   South  101.0
Carl     West    NaN
Tony     West  103.0
Mo       East    NaN
Sally   South  202.0
Fred      NaN   82.0


Merge полезен, когда мы не хотим привязываться к индексам. Скажем, мы хотим знать, сколько, в процентном отношении,  каждый сотрудник внес в продажи в своем регионе. Мы можем использовать groupby для суммирования всех продаж в каждом уникальном регионе. В приведенном ниже коде reset_index используется, чтобы превратить регион из индекса в обычную колонку.

In [11]:
grouped_df = joined_df_merge.groupby(by='region').sum()
grouped_df.reset_index(inplace=True)
print(grouped_df)

  region  sales
0   East  380.0
1  North    0.0
2  South  303.0
3   West  103.0


Теперь остается смержить join_df_merge с grouped_df, используя столбец region. Мы должны указать суффикс, потому что оба наших блока данных (которые мы объединяем) содержат столбец с названием sales. Входные суффиксы добавляют указанные строки к меткам столбцов с одинаковыми именами в обоих датафреймах (это удобно, потому что ничего не перепутается). В нашем случае, поскольку столбец продаж второго датафрейма фактически отражает продажи во всем регионе, мы можем добавить суффикс «_region».

In [12]:
employee_contrib = joined_df_merge.merge(grouped_df, how='left', 
                                         left_on='region', 
                                         right_on='region',
                                         suffixes=('','_region'))
print(employee_contrib)

  region  sales  sales_region
0  North    NaN           0.0
1    NaN    NaN           NaN
2   East  380.0         380.0
3  South  101.0         303.0
4   West    NaN         103.0
5   West  103.0         103.0
6   East    NaN         380.0
7  South  202.0         303.0
8    NaN   82.0           NaN


ЭЭЭЭЭ, куда индекс исчез? 

Используем set_index, чтобы вернуть его! (иначе мы не узнаем, какому сотруднику соответствует какая строка):

In [13]:
employee_contrib = employee_contrib.set_index(joined_df_merge.index)
print(employee_contrib)

       region  sales  sales_region
Archie  North    NaN           0.0
HanWei    NaN    NaN           NaN
Randy    East  380.0         380.0
Ellen   South  101.0         303.0
Carl     West    NaN         103.0
Tony     West  103.0         103.0
Mo       East    NaN         380.0
Sally   South  202.0         303.0
Fred      NaN   82.0           NaN


Теперь у нас есть исходный столбец продаж и новый столбец sales_region, в котором указывается общий объем продаж в регионе. Давайте посчитаем процент продаж каждого сотрудника, а затем очистим наш датафрейм, отбросив наблюдения, без региона (Fred и HanWei), и заполняя NaN в столбце продаж нулями.

In [14]:
# Drop NAs in region column
employee_contrib = employee_contrib.dropna(subset=['region'])

# Fill NAs in sales column with 0
employee_contrib = employee_contrib.fillna({'sales': 0})

employee_contrib['%_of_sales'] = employee_contrib['sales']/employee_contrib['sales_region']

print(employee_contrib[['region','sales','%_of_sales']].sort_values(by=['region','%_of_sales']))

       region  sales  %_of_sales
Mo       East    0.0    0.000000
Randy    East  380.0    1.000000
Archie  North    0.0         NaN
Ellen   South  101.0    0.333333
Sally   South  202.0    0.666667
Carl     West    0.0    0.000000
Tony     West  103.0    1.000000
